# A Wright-Fisher simulation implemented in C via Cython.

OMG!

We would use GSL via CythonGSL, but that would require a GPL license for this notebook, but we're doing CCBY.

In [1]:
%load_ext Cython

In [2]:
import msprime
import numpy as np

/Users/kevin/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [37]:
%%cython -3 -lgsl -lgslcblas -lm

import msprime
import numpy as np
from collections import namedtuple
import pickle
cimport numpy as np
from cython.view cimport array as cvarray
from libc.stdlib cimport malloc, realloc, free
from libc.stdint cimport int32_t, uint32_t

from cython_gsl.gsl_rng cimport gsl_rng
from cython_gsl.gsl_rng cimport gsl_rng_mt19937
from cython_gsl.gsl_rng cimport gsl_rng_alloc
from cython_gsl.gsl_rng cimport gsl_rng_free
from cython_gsl.gsl_rng cimport gsl_rng_set
from cython_gsl.gsl_rng cimport gsl_rng_uniform
from cython_gsl.gsl_random cimport gsl_ran_flat
from cython_gsl.gsl_random cimport gsl_ran_poisson
from cython_gsl.gsl_vector cimport gsl_vector_view
from cython_gsl.gsl_vector cimport gsl_vector_view_array
from cython_gsl.gsl_vector cimport gsl_vector_minmax

cdef int32_t * malloc_int32_t(size_t n):
    return <int32_t*>malloc(n*sizeof(int32_t))

cdef int32_t * realloc_int32_t(void * x, size_t n):
    return <int32_t*>realloc(x,n*sizeof(int32_t))

cdef double * malloc_double(size_t n):
    return <double*>malloc(n*sizeof(double))

cdef double * realloc_double(double * x, size_t n):
    return <double*>realloc(<double *>x,n*sizeof(double))

cdef struct Mutations:
    double * pos
    int32_t * time
    int32_t * node
    size_t next_mutation, capacity
    
cdef int init_Mutations(Mutations * m):
    m.next_mutation = 0
    m.capacity = 10000
    m.pos = malloc_double(m.capacity)
    if m.pos == NULL:
        return -1
    m.time = malloc_int32_t(m.capacity)
    if m.time == NULL:
        return -1
    m.node = malloc_int32_t(m.capacity)
    if m.node == NULL:
        return -1
    return 0

cdef int realloc_Mutations(Mutations * m):
    m.capacity *= 2
    m.pos = realloc_double(m.pos,
                          m.capacity)
    if m.pos == NULL:
        return -1
    m.time = realloc_int32_t(m.time,
                            m.capacity)
    if m.time == NULL:
        return -1
    m.node = realloc_int32_t(m.node,
                            m.capacity)
    if m.node == NULL:
        return -1
    return 0

cdef void free_Mutations(Mutations * m):
    free(m.pos)
    free(m.time)
    free(m.node)
    m.next_mutation = 0
    m.capacity = 10000
    
cdef int add_mutation(double pos,
                     int32_t generation,
                     int32_t node,
                     Mutations * m):
    cdef int rv = 0
    if m.next_mutation+1 >= m.capacity:
        rv = realloc_Mutations(m)
        if rv != 0:
            return rv
    m.pos[m.next_mutation] = pos
    m.time[m.next_mutation] = generation
    m.node[m.next_mutation] = node
    m.next_mutation+=1
    return rv
    
cdef struct Nodes:
    double * time
    size_t next_node, capacity
    
cdef int init_Nodes(Nodes * n):
    n.next_node = 0
    n.capacity = 10000
    n.time = malloc_double(n.capacity)
    if n.time == NULL:
        return -1
    return 0

cdef int realloc_Nodes(Nodes * n):
    n.capacity *= 2
    n.time = realloc_double(n.time,
                            n.capacity)
    if n.time == NULL:
        return -1
    return 0
    
cdef void free_Nodes(Nodes * n):
    if n.time != NULL:
        free(n.time)
    n.next_node = 0
    n.capacity = 10000

cdef int add_node(double t, Nodes *n):
    cdef int rv = 0
    if n.next_node >= n.capacity:
        rv = realloc_Nodes(n)
        if rv != 0:
            return rv
    n.time[n.next_node] = t
    n.next_node+=1
    return rv
    
cdef struct Edges:
    double *left
    double *right
    int32_t *parent
    int32_t *child
    size_t next_edge, capacity
    
cdef int init_Edges(Edges * e):
    e.next_edge = 0
    e.capacity = 10000
    e.left = malloc_double(e.capacity)
    if e.left == NULL:
        return -1
    e.right = malloc_double(e.capacity)
    if e.right == NULL:
        return -1
    e.parent = malloc_int32_t(e.capacity)
    if e.parent == NULL:
        return -1
    e.child = malloc_int32_t(e.capacity)
    if e.child == NULL:
        return -1
    return 0
   
cdef int realloc_Edges(Edges * e):
    e.capacity *= 2
    e.left = realloc_double(e.left,e.capacity)
    if e.left == NULL:
        return -1
    e.right = realloc_double(e.right,e.capacity)
    if e.right == NULL:
        return -1
    e.parent = realloc_int32_t(e.parent,e.capacity)
    if e.parent == NULL:
        return -1
    e.child = realloc_int32_t(e.child,e.capacity)
    if e.child == NULL:
        return -1
    return 0

cdef void free_Edges(Edges * e):
    free(e.left)
    free(e.right)
    free(e.parent)
    free(e.child)
    e.next_edge = 0
    e.capacity = 10000
    
cdef int add_edge(double left, double right,
             int32_t parent, int32_t child,
             Edges * edges):
    cdef int rv=0
    if edges.next_edge+1 >= edges.capacity:
        rv = realloc_Edges(edges)
        if rv != 0:
            return rv
        
    edges.left[edges.next_edge] = left
    edges.right[edges.next_edge] = right
    edges.parent[edges.next_edge] = parent
    edges.child[edges.next_edge] = child
    edges.next_edge += 1
    return rv

cdef struct Tables:
    Nodes nodes
    Edges edges
    Mutations mutations
    gsl_rng * rng
    
cdef int init_Tables(Tables * t, int seed):
    cdef int rv = 0
    rv = init_Nodes(&t.nodes)
    if rv != 0:
        return rv
    rv = init_Edges(&t.edges)
    if rv != 0:
        return rv
    rv = init_Mutations(&t.mutations)
    if rv != 0:
        return rv
    t.rng = gsl_rng_alloc(gsl_rng_mt19937)
    if t.rng == NULL:
        return -1
    gsl_rng_set(t.rng, seed)
    return rv

cdef void free_Tables(Tables * t):
    free_Nodes(&t.nodes)
    free_Edges(&t.edges)
    free_Mutations(&t.mutations)
    gsl_rng_free(t.rng)
    
cdef int infsites(double mu, int32_t generation,
                  int32_t next_offspring_index,
                  Tables * tables,
                  dict lookup):
    cdef unsigned nmut = gsl_ran_poisson(tables.rng, mu)
    cdef unsigned i = 0
    cdef double pos
    cdef int rv = 0
    for i in range(nmut):
        pos = gsl_rng_uniform(tables.rng)
        while pos in lookup:
            pos = gsl_rng_uniform(tables.rng)
        rv = add_mutation(pos,
                         generation,
                         next_offspring_index,
                         &tables.mutations)
        if rv != 0:
            return rv
        lookup[pos] = True
    return rv

cdef int poisson_recombination(double r,
                               size_t pg1, size_t pg2,
                                int32_t next_offspring_id,
                               Tables * tables):
    cdef unsigned nbreaks = gsl_ran_poisson(tables.rng, r)
    cdef list b = []
    cdef size_t lenb
    cdef unsigned i = 0,j=0
    cdef double x
    # cdef list pgams
    cdef int rv = 0
    cdef double left,right
    cdef int32_t p
    if nbreaks == 0:
        # The parent passes the entire region onto the child
        rv = add_edge(0.0,1.0,pg1,
                      next_offspring_id,&tables.edges)
        if rv != 0:
            return rv
    else:
        while i < nbreaks:
            x = gsl_rng_uniform(tables.rng)
            while x in b:
                x = gsl_rng_uniform(tables.rng)
            b.append(x)
            i += 1
        b.sort()
        b.append(1.0)

        print("breakpoints = ",b,j)
        print("pgams=",pg1,pg2)
        if b[0] != 0.0:
            b.insert(0,0.0)
        else:
            j=1
            print(pg1,pg2)
            pg1,pg2 = pg2,pg1
            print(pg1,pg2)
            #parent_indexes = (parent_indexes[1], parent_indexes[0])
        
        #pgams = list([*tuple(i for i in parent_indexes)]*int(len(b)/2))

        #for left,right in zip(b[0:len(b)-1],b[1:len(b)]):
        lenb = len(b)-1
        print("lens:",lenb,len(b),b[0],j)
        for i in range(lenb):
            left=b[i]
            right=b[i+1]
            print(pg1,pg2,next_offspring_id,left,right)
            rv = add_edge(left,right,pg1,
                          next_offspring_id,&tables.edges)
            if rv != 0:
                return rv
            pg1,pg2 = pg2,pg1
            if j == 1:
                print(pg1,pg2)
    #if j==1:
    #    print("//")
    return 0

MutationMetadata = namedtuple('MutationMetadata',['pos','origin'])

cdef int simplify(Tables * tables, 
            double dt,
            object nodes,
            object edges,
            object sites,
            object mutations):
    cdef int rv = 0,gap
    
    cdef size_t i,nedges=len(edges)
    cdef np.ndarray[double,ndim=1] dview,lview,rview
    cdef np.ndarray[int32_t,ndim=1] pview,cview
    # Reverse time for our new nodes
    cdef gsl_vector_view vt
    vt = gsl_vector_view_array(tables.nodes.time,<size_t>tables.nodes.next_node)
    cdef double tmax,tmin
    gsl_vector_minmax(&vt.vector,&tmin,&tmax)
    # if tables.nodes.next_node > 0:
    #     g=tables.nodes.time[tables.nodes.next_node-1]
    for i in range(tables.nodes.next_node):
        tables.nodes.time[i] = -1.0*(tables.nodes.time[i]-tmax)
    gsl_vector_minmax(&vt.vector,&tmin,&tmax)
    
    
    nodes.set_columns(time=nodes.time+dt,flags=nodes.flags)
    dview = np.asarray(<double[:tables.nodes.next_node]>tables.nodes.time)
    gap=nodes.time.min()-tmax
    print(nodes.time.min(),nodes.time.max(),dview.min(),dview.max(),tmax,gap,dt)
    if gap != 1:
        return -1
    nodes.append_columns(time=dview,
                        flags=np.ones(tables.nodes.next_node,dtype=np.uint32))
    
    
    lview = np.asarray(<double[:tables.edges.next_edge]>tables.edges.left)
    rview = np.asarray(<double[:tables.edges.next_edge]>tables.edges.right)
    pview = np.asarray(<int32_t[:tables.edges.next_edge]>tables.edges.parent)
    cview = np.asarray(<int32_t[:tables.edges.next_edge]>tables.edges.child)
    edges.append_columns(left=lview,
                        right=rview,
                        parent=pview,
                        child=cview)
    
    # Disclaimer: we will use .add_row here 
    # to keep the code as simple as possible.
    # However, things will go MUCH faster
    # if we use .append_columns, whose syntax
    # is not going to be pleasant here :)
    try:
        for i in range(tables.mutations.next_mutation):
            sites.add_row(position=tables.mutations.pos[i],
                         ancestral_state='0')
            # md = MutationMetadata(float(tables.mutations.pos[i]),
            #                       int(tables.mutations.time[i]))
            # print(md)
            #mdp = pickle.dumps(md)
            #print(type(mdp))
            mutations.add_row(site=<uint32_t>(len(sites)-1),
                              node=<uint32_t>tables.mutations.node[i],
                              derived_state='1')
                              # metadata=md)
    except:
        return -1
    
    for ei in edges:
        assert(ei.parent < len(nodes))
        assert(ei.child < len(nodes))
    msprime.sort_tables(nodes=nodes,edges=edges,
                       sites=sites,mutations=mutations,
                       edge_start=nedges)
    
    samples = np.where(nodes.time == 0)[0]
    print(len(nodes),len(edges),len(sites),len(mutations),len(samples))
    print(samples) 
    msprime.simplify_tables(samples=samples.tolist(),
                            nodes=nodes,
                           edges=edges,
                            sites=sites,
                            mutations=mutations)
    
    # "clear" our temp containers
    tables.nodes.next_node = 0
    tables.mutations.next_mutation = 0
    tables.edges.next_edge = 0
                          
    return rv

def evolve(int N, int ngens, double theta, double rho, int gc, int seed):
    nodes = msprime.NodeTable()
    edges = msprime.EdgeTable()
    sites = msprime.SiteTable()
    mutations = msprime.MutationTable()
    
    cdef double mu = theta/<double>(4*N)
    cdef double r = rho/<double>(4*N)
    
    cdef int rv
    cdef size_t i, generation
    cdef Tables tables
    rv = init_Tables(&tables, seed)
    if rv != 0:
        free_Tables(&tables)
        raise RuntimeError("could not initialize tables")
        
    for i in range(2*<size_t>N):
        nodes.add_row(time=0.0,
                      flags=msprime.NODE_IS_SAMPLE)
        
    
    cdef int32_t next_offspring_index, first_parental_index
    next_offspring_index = len(nodes)
    first_parental_index = 0
    cdef size_t parent1, parent2,pindex
    cdef int32_t p1g1, p1g2, p2g1, p2g2
    cdef dict lookup = {}
    cdef size_t last_gen_gc = 0
    for generation in range(<size_t>(ngens)):
        print("gen = ",generation)
        if generation>0 and generation%gc == 0.0:
            print("entering GC",generation,gc)
            x=np.asarray(<double[:tables.nodes.next_node]>tables.nodes.time)
            print(x)
            rv = simplify(&tables,
                         generation-last_gen_gc,
                         nodes,edges,sites,mutations)
            if rv != 0:
                raise RuntimeError("simplification error")
            last_gen_gc=generation
            next_offspring_index = len(nodes)
            first_parental_index = 0
        else:
            first_parental_index = next_offspring_index - 2*N
            print("idx check:",len(nodes),tables.nodes.next_node,next_offspring_index,first_parental_index)
            
        for pindex in range(0,2*N,2):
            parent1=<size_t>gsl_ran_flat(tables.rng,0.0,<double>N)
            parent2=<size_t>gsl_ran_flat(tables.rng,0.0,<double>N)
            p1g1 = first_parental_index + 2*parent1
            p1g2 = p1g1 + 1
            p2g1 = first_parental_index + 2*parent2
            p2g2 = p2g1 + 1
            
            if gsl_rng_uniform(tables.rng) < 0.5:
                p1g1, p1g2 = p1g2, p1g1
            if gsl_rng_uniform(tables.rng) < 0.5:
                p2g1, p2g2 = p2g2, p2g1
                
            rv = poisson_recombination(r,p1g1,p1g2,
                                      next_offspring_index,
                                      &tables)
            if rv != 0:
                free_Tables(&tables)
                raise RuntimeError("error during recombination")
                
            rv = infsites(mu,generation+1,
                          next_offspring_index,
                         &tables,lookup)
            
            if rv != 0:
                free_Tables(&tables)
                raise RuntimeError("error during mutation")
                
            rv = add_node(generation+1, &tables.nodes)
            if rv != 0:
                free_Tables(&tables)
                raise RuntimeError("error during adding nodes")
                
            next_offspring_index += 1
                
            rv = poisson_recombination(r,p2g1,p2g2,
                                      next_offspring_index,
                                      &tables)
            if rv != 0:
                free_Tables(&tables)
                raise RuntimeError("error during recombination")
                
            rv = infsites(mu,generation+1,
                          next_offspring_index,
                         &tables,lookup)
            
            if rv != 0:
                free_Tables(&tables)
                raise RuntimeError("error during mutation")
                
            rv = add_node(generation+1, &tables.nodes)
            if rv != 0:
                free_Tables(&tables)
                raise RuntimeError("error during adding nodes")
                
            next_offspring_index += 1
        print('finished generation',generation)   
            #assert(next_offspring_index <= len(nodes)+tables.nodes.next_node)
            #print(next_offspring_index,len(nodes),tables.nodes.next_node)
        # first_parental_index += 2*N
        
    if tables.nodes.next_node > 0:
        print("final values = ",generation,last_gen_gc,
             tables.nodes.time[tables.nodes.next_node-1])
        rv=simplify(&tables,
                   generation-last_gen_gc,
                    nodes,edges,sites,mutations
                   )
    
    return msprime.load_tables(nodes=nodes,edges=edges)

In [38]:
# %%time
# evolve(1000, 11, 500.0, 500.0, 3, 42)

In [39]:
evolve(100,11,19,10,3,42)

gen =  0
idx check: 200 0 200 0
breakpoints =  [0.7722447717096657, 1.0] 0
pgams= 59 58
lens: 2 3 0.0 0
59 58 233 0.0 0.7722447717096657
58 59 233 0.7722447717096657 1.0
breakpoints =  [0.140084013575688, 1.0] 0
pgams= 15 14
lens: 2 3 0.0 0
15 14 325 0.0 0.140084013575688
14 15 325 0.140084013575688 1.0
breakpoints =  [0.4126176794525236, 1.0] 0
pgams= 150 151
lens: 2 3 0.0 0
150 151 388 0.0 0.4126176794525236
151 150 388 0.4126176794525236 1.0
finished generation 0
gen =  1
idx check: 200 200 400 200
breakpoints =  [0.7533781833481044, 1.0] 0
pgams= 362 363
lens: 2 3 0.0 0
362 363 414 0.0 0.7533781833481044
363 362 414 0.7533781833481044 1.0
breakpoints =  [0.07379656378179789, 1.0] 0
pgams= 378 379
lens: 2 3 0.0 0
378 379 428 0.0 0.07379656378179789
379 378 428 0.07379656378179789 1.0
breakpoints =  [0.5554317058995366, 1.0] 0
pgams= 362 363
lens: 2 3 0.0 0
362 363 456 0.0 0.5554317058995366
363 362 456 0.5554317058995366 1.0
breakpoints =  [0.4926180960610509, 1.0] 0
pgams= 251 250


LibraryError: Edges must be listed in (time[parent], child, left) order; time[parent] order violated

breakpoints =  [0.4075842092279345, 1.0] 0
pgams= 114 115
lens: 2 3 0.0 0
114 115 910 0.0 0.4075842092279345
115 114 910 0.4075842092279345 1.0
breakpoints =  [0.6768212930765003, 1.0] 0
pgams= 191 190
lens: 2 3 0.0 0
191 190 924 0.0 0.6768212930765003
190 191 924 0.6768212930765003 1.0
breakpoints =  [0.6034930991008878, 1.0] 0
pgams= 168 169
lens: 2 3 0.0 0
168 169 947 0.0 0.6034930991008878
169 168 947 0.6034930991008878 1.0
breakpoints =  [0.12555021652951837, 1.0] 0
pgams= 154 155
lens: 2 3 0.0 0
154 155 1008 0.0 0.12555021652951837
155 154 1008 0.12555021652951837 1.0
breakpoints =  [0.5695397835224867, 1.0] 0
pgams= 198 199
lens: 2 3 0.0 0
198 199 1052 0.0 0.5695397835224867
199 198 1052 0.5695397835224867 1.0
breakpoints =  [0.7573102093301713, 1.0] 0
pgams= 139 138
lens: 2 3 0.0 0
139 138 1059 0.0 0.7573102093301713
138 139 1059 0.7573102093301713 1.0
breakpoints =  [0.04491130239330232, 1.0] 0
pgams= 181 180
lens: 2 3 0.0 0
181 180 1073 0.0 0.04491130239330232
180 181 1073 0.

Exception ignored in: '_cython_magic_507c44d3c03dd7fae47131bd4db64a78.simplify'
Traceback (most recent call last):
  File "/Users/kevin/anaconda3/lib/python3.5/site-packages/msprime/tables.py", line 1279, in simplify_tables
    return _msprime.simplify_tables(**kwargs)
_msprime.LibraryError: Edges must be listed in (time[parent], child, left) order; time[parent] order violated


RuntimeError: simplification error